In [1]:
from selenium import webdriver #to automate the navigating within the browser
from selenium.webdriver.chrome.service import service
from selenium.webdriver.common.keys    import Keys
from selenium.webdriver.support.ui     import Select
from selenium.webdriver.support.ui     import WebDriverWait 
from selenium.webdriver.common.by      import By
from selenium.webdriver.support        import expected_conditions as EC
from selenium.webdriver.chrome.options import Options #to use properties of the chrome webbrowser
from selenium.webdriver.remote.command import Command # Use to check whether the web driver is active

import requests      # Requests data from an API
import json          # Handles JSON object
import jmespath      # Allows you to mor easily navigate JSON objects
import pandas as pd  # Handles data frames



In [28]:
url_ai_works = \
      "https://api.openalex.org/works?filter=title.search:artificial intelligence"

In [29]:
search_results_ai_works = requests.\
    get(url_ai_works).json()

In [31]:
with open('json_files/convert_search_results_ai_works.json', 'w') as file: 
    file.write(json.dumps(search_results_ai_works, indent =4))

In [30]:
total_ai_works = search_results_ai_works['meta']['count']
print("The number of works with 'artificial intelligence' in the title is ", total_ai_works)

The number of works with 'artificial intelligence' in the title is  138407


In [23]:
count_list = []
for i in search_results['results']:
    count_list.append(i['title'])

print(len(count_list))

25


In [24]:
test = "https://api.openalex.org/institutions?search=stanford"

In [26]:
search_results2 = requests.\
    get(test).json()

In [27]:
print(search_results2)

{'meta': {'count': 11, 'db_response_time_ms': 32, 'page': 1, 'per_page': 25, 'groups_count': None}, 'results': [{'id': 'https://openalex.org/I97018004', 'ror': 'https://ror.org/00f54p054', 'display_name': 'Stanford University', 'relevance_score': 329652.6, 'country_code': 'US', 'type': 'education', 'type_id': 'https://openalex.org/institution-types/education', 'lineage': ['https://openalex.org/I97018004'], 'homepage_url': 'https://www.stanford.edu', 'image_url': 'https://commons.wikimedia.org/w/index.php?title=Special:Redirect/file/Stanford%20wordmark%20%282012%29.svg', 'image_thumbnail_url': 'https://commons.wikimedia.org/w/index.php?title=Special:Redirect/file/Stanford%20wordmark%20%282012%29.svg&width=300', 'display_name_acronyms': ['SU'], 'display_name_alternatives': ['Universidad Stanford', 'Leland Stanford Junior University'], 'repositories': [], 'works_count': 457515, 'cited_by_count': 26508244, 'summary_stats': {'2yr_mean_citedness': 5.384909138793785, 'h_index': 1486, 'i10_ind